# Working with AI Agents

This notebook introduces the concept of AI agents and how to build them using the OpenAI SDK. We'll cover what agents are, how they differ from simple chatbots, and how to create agents that can use tools, make decisions, and perform complex tasks autonomously.

---


## Table of Contents

1. [What Are AI Agents?](#what-are-ai-agents)
2. [Setting Up the OpenAI SDK](#setting-up-the-openai-sdk)
3. [Creating Your First Agent](#creating-your-first-agent)
4. [Agents with Function Calling](#agents-with-function-calling)
5. [Building Multi-Step Agent Workflows](#building-multi-step-agent-workflows)
6. [Best Practices and Patterns](#best-practices-and-patterns)
---


## What Are AI Agents?

An **AI agent** is an AI system that can autonomously perform tasks by:
- **Understanding goals** - Interpreting user requests and breaking them into steps
- **Making decisions** - Choosing which actions to take based on context
- **Using tools** - Calling functions, APIs, or other services to accomplish tasks
- **Iterating** - Refining its approach based on results until the goal is achieved

**Agents vs. Simple Chatbots:**
- **Chatbots** respond to individual messages with text
- **Agents** can plan multi-step workflows, use external tools, and persist state across interactions

**Common Use Cases:**
- <u>Research assistants</u> that search the web and synthesize information
- <u>Code agents</u> that write, test, and debug programs
- <u>Data analysis</u> agents that query databases and generate reports
- <u>Task automation</u> agents that interact with APIs and services

---


## Setting Up the OpenAI SDK

Before building agents, we need to install the OpenAI Python SDK and configure our API key.


In [ ]:
# Install required packages
# Run this in your terminal: uv add openai python-dotenv

import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables from .env file
load_dotenv()

# Initialize the OpenAI client
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

print("OpenAI client initialized successfully!")


**Important:** Make sure you have a `.env` file in your project root with your OpenAI API key:
 
 ```
 OPENAI_API_KEY=your-api-key-here
 ```
 
 **File Structure:**
 ```
 your-project/
 ├── .env                     <- Your API key goes here
 ├── pyproject.toml
 └── 01-SWE-Fundamentals/
     └── 01-Python-for-AI/
         └── WorkingWithAgents.ipynb
 ```
 
 Never commit your API key to version control!
 
 ---


### Creating Your First Agent

Let's start with a simple agent that can have a conversation and maintain context. This basic agent uses the Chat Completions API with a system message that defines its role and behavior.


In [ ]:
def simple_agent(user_message: str, conversation_history: list = None) -> str:
    """
    A simple agent that maintains conversation context.
    
    Args:
        user_message: The user's message
        conversation_history: List of previous messages in the format:
            [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]
    
    Returns:
        The agent's response
    """
    if conversation_history is None:
        conversation_history = []
    
    # System message defines the agent's role and behavior
    system_message = {
        "role": "system",
        "content": "You are a helpful AI assistant. Be concise, accurate, and friendly."
    }
    
    # Build the messages list
    messages = [system_message] + conversation_history + [
        {"role": "user", "content": user_message}
    ]
    
    # Call the API
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Using a cost-effective model
        messages=messages,
        temperature=0.7
    )
    
    return response.choices[0].message.content

# Example usage
response = simple_agent("What is Python?")
print("Agent:", response)


In [ ]:
# Example with conversation history
history = [
    {"role": "user", "content": "My name is Alice"},
    {"role": "assistant", "content": "Nice to meet you, Alice! How can I help you today?"}
]

response = simple_agent("What's my name?", history)
print("Agent:", response)


---

### Agents with Function Calling

The real power of agents comes from their ability to use **tools** (functions). This allows agents to:
- Perform calculations
- Search databases
- Call APIs
- Interact with external services
- Make decisions based on real data

Let's create an agent that can use a calculator tool:


In [ ]:
import json
import math

def calculator(expression: str) -> float:
    """
    Evaluates a mathematical expression safely.
    
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 2", "sqrt(16)")
    
    Returns:
        The result of the calculation
    """
    # Only allow safe math operations
    allowed_names = {
        k: v for k, v in math.__dict__.items() if not k.startswith("__")
    }
    allowed_names.update({"abs": abs, "round": round, "min": min, "max": max})
    
    try:
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return float(result)
    except Exception as e:
        return f"Error: {str(e)}"

# Test the calculator function
print("Test:", calculator("2 + 2"))
print("Test:", calculator("sqrt(16)"))


In [ ]:
def agent_with_tools(user_message: str, conversation_history: list = None) -> str:
    """
    An agent that can use tools (functions) to accomplish tasks.
    """
    if conversation_history is None:
        conversation_history = []
    
    # Define available tools
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculator",
                "description": "Evaluates a mathematical expression. Use this when the user asks for calculations.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate (e.g., '2 + 2', 'sqrt(16)')"
                        }
                    },
                    "required": ["expression"]
                }
            }
        }
    ]
    
    system_message = {
        "role": "system",
        "content": "You are a helpful math assistant. When users ask for calculations, use the calculator tool."
    }
    
    messages = [system_message] + conversation_history + [
        {"role": "user", "content": user_message}
    ]
    
    # First API call - the model may request to use a tool
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto"  # Let the model decide when to use tools
    )
    
    message = response.choices[0].message
    messages.append(message)
    
    # If the model wants to use a tool, execute it and send the result back
    while message.tool_calls:
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            # Execute the function
            if function_name == "calculator":
                function_response = calculator(function_args["expression"])
            else:
                function_response = "Unknown function"
            
            # Add the tool result to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(function_response)
            })
        
        # Get the model's response to the tool results
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools
        )
        message = response.choices[0].message
        messages.append(message)
    
    return message.content

# Example usage
result = agent_with_tools("What is 15 * 23 + sqrt(144)?")
print("Agent:", result)


---

### Building Multi-Step Agent Workflows

Real-world agents often need to perform multiple steps to accomplish a goal. Let's create a research agent that can:
1. Break down complex questions into steps
2. Use multiple tools
3. Synthesize information from multiple sources


In [ ]:
import datetime

def get_current_time() -> str:
    """Returns the current date and time."""
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def search_knowledge_base(query: str) -> str:
    """
    Simulates searching a knowledge base.
    In a real application, this would query a database or vector store.
    """
    # Simulated knowledge base
    knowledge = {
        "python": "Python is a high-level programming language known for its simplicity and readability.",
        "ai": "Artificial Intelligence (AI) refers to systems that can perform tasks typically requiring human intelligence.",
        "agents": "AI agents are autonomous systems that can make decisions and use tools to accomplish goals."
    }
    
    query_lower = query.lower()
    results = []
    for key, value in knowledge.items():
        if key in query_lower:
            results.append(f"{key.capitalize()}: {value}")
    
    return "\n".join(results) if results else "No relevant information found."

def web_search(query: str) -> str:
    """
    Simulates a web search.
    In a real application, this would call a search API like Google or Bing.
    """
    return f"[Simulated] Search results for '{query}': Found 5 relevant articles about {query}."


In [ ]:
def research_agent(user_query: str, max_iterations: int = 5) -> str:
    """
    A research agent that can use multiple tools and perform multi-step research.
    """
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Gets the current date and time. Use when the user asks about time or dates.",
                "parameters": {"type": "object", "properties": {}}
            }
        },
        {
            "type": "function",
            "function": {
                "name": "search_knowledge_base",
                "description": "Searches the internal knowledge base for information. Use for general questions.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query"
                        }
                    },
                    "required": ["query"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "web_search",
                "description": "Searches the web for current information. Use for recent events or topics not in the knowledge base.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query"
                        }
                    },
                    "required": ["query"]
                }
            }
        }
    ]
    
    system_message = {
        "role": "system",
        "content": """You are a research assistant. When answering questions:
1. First try the knowledge base for general information
2. Use web search for current events or specific details
3. Use get_current_time for time-related questions
4. Synthesize information from multiple sources into a comprehensive answer"""
    }
    
    messages = [system_message, {"role": "user", "content": user_query}]
    iterations = 0
    
    while iterations < max_iterations:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        message = response.choices[0].message
        messages.append(message)
        
        # If no tool calls, we're done
        if not message.tool_calls:
            break
        
        # Execute tool calls
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            if function_name == "get_current_time":
                function_response = get_current_time()
            elif function_name == "search_knowledge_base":
                function_response = search_knowledge_base(function_args["query"])
            elif function_name == "web_search":
                function_response = web_search(function_args["query"])
            else:
                function_response = "Unknown function"
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": function_response
            })
        
        iterations += 1
    
    return messages[-1].content

# Example usage
result = research_agent("What is Python and what are its main features?")
print("Research Agent:", result)


---

### Best Practices and Patterns

When building production-ready agents, consider these best practices:

#### 1. Error Handling
Always handle errors gracefully and provide meaningful feedback to users.


In [ ]:
def robust_agent(user_message: str) -> str:
    """An agent with proper error handling."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": user_message}
            ],
            temperature=0.7,
            max_tokens=500
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"I apologize, but I encountered an error: {str(e)}. Please try again."

# Example with error handling
result = robust_agent("Hello!")
print(result)


#### 2. Token Management
Monitor token usage to control costs and stay within limits.


In [ ]:
def agent_with_token_tracking(user_message: str) -> tuple[str, dict]:
    """An agent that tracks token usage."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_message}
        ]
    )
    
    usage = {
        "prompt_tokens": response.usage.prompt_tokens,
        "completion_tokens": response.usage.completion_tokens,
        "total_tokens": response.usage.total_tokens
    }
    
    return response.choices[0].message.content, usage

# Example
response, usage = agent_with_token_tracking("Explain AI in one sentence.")
print("Response:", response)
print("Token Usage:", usage)


#### 3. Streaming Responses
For better user experience, stream responses for long-running tasks.


In [ ]:
def streaming_agent(user_message: str):
    """An agent that streams responses as they're generated."""
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_message}
        ],
        stream=True
    )
    
    full_response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            full_response += content
            print(content, end="", flush=True)  # Print as it streams
    
    print()  # New line at the end
    return full_response

# Example (uncomment to test)
# streaming_agent("Tell me a short story about AI.")


#### 4. Agent Classes
Organize your agents into classes for better code structure and reusability.


In [ ]:
class Agent:
    """A reusable agent class."""
    
    def __init__(self, name: str, system_prompt: str, model: str = "gpt-4o-mini"):
        self.name = name
        self.system_prompt = system_prompt
        self.model = model
        self.conversation_history = []
    
    def chat(self, user_message: str) -> str:
        """Send a message to the agent and get a response."""
        messages = [
            {"role": "system", "content": self.system_prompt}
        ] + self.conversation_history + [
            {"role": "user", "content": user_message}
        ]
        
        response = client.chat.completions.create(
            model=self.model,
            messages=messages
        )
        
        assistant_message = response.choices[0].message.content
        
        # Update conversation history
        self.conversation_history.append({"role": "user", "content": user_message})
        self.conversation_history.append({"role": "assistant", "content": assistant_message})
        
        return assistant_message
    
    def reset(self):
        """Reset the conversation history."""
        self.conversation_history = []

# Example usage
math_tutor = Agent(
    name="Math Tutor",
    system_prompt="You are a patient math tutor. Explain concepts step-by-step."
)

response = math_tutor.chat("What is the Pythagorean theorem?")
print(f"{math_tutor.name}: {response}")


---

## Summary

In this notebook, we've covered:

- **What agents are** - Autonomous AI systems that can use tools and make decisions
- **Basic agent creation** - Simple conversational agents with context
- **Function calling** - Agents that can use tools to accomplish tasks
- **Multi-step workflows** - Complex agents that perform research and synthesis
- **Best practices** - Error handling, token management, streaming, and code organization

**Next Steps:**
- Experiment with different tools and functions
- Build agents for specific use cases (code generation, data analysis, etc.)
- Explore more advanced patterns like agent orchestration and multi-agent systems
- Consider using frameworks like LangChain or AutoGPT for production applications

**Resources:**
- [OpenAI API Documentation](https://platform.openai.com/docs)
- [Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)
- [Best Practices](https://platform.openai.com/docs/guides/best-practices)
